In [151]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [152]:
DATA_FOLDER = "../data"
ABUNDANCE_FILE = "abundance_with_unique.csv"
ABUNDANCE_START = 213

LAYERS = [1024, 512, 256, 128, 64]
BATCH_SIZE = 32

In [153]:
abundance_data = pd.read_csv(os.path.join(DATA_FOLDER, ABUNDANCE_FILE), low_memory=False)

metadata = abundance_data.iloc[:, :212]
abundance = abundance_data.iloc[:, 212:]

In [154]:
diseases_list = list(metadata['disease'].unique())

# Healthy = ['n', 'nd', ' -', 'leaness', 'n_relative', '-'] : Change this in the list and dict to 'healthy'
diseases_list_new = ['healthy' if x in ['n', 'nd', ' -', 'leaness', 'n_relative', '-'] else x for x in diseases_list]

# Change 'y' to psoriasis
diseases_list_new = ['psoriasis' if x == 'y' else x for x in diseases_list_new]

diseases_list_unique = list(set(diseases_list_new))

# Create new column in metadata with the new disease names
metadata['disease_new'] = metadata['disease'].apply(lambda x: 'healthy' if x in ['n', 'nd', ' -', 'leaness', 'n_relative', '-'] else 'psoriasis' if x == 'y' else x)

# put disease_new column in the second column after UniqueID
metadata = metadata[['UniqueID', 'disease_new'] + [col for col in metadata.columns if col not in ['UniqueID', 'disease_new']]]

new_abundance_data = pd.concat([metadata, abundance], axis=1)

del metadata, abundance

# print frequency of each disease
new_abundance_data['disease_new'].value_counts()

disease_new
healthy                       2692
t2d                            223
obesity                        164
ibd_ulcerative_colitis         148
cirrhosis                      118
stec2-positive                  52
impaired_glucose_tolerance      49
cancer                          48
psoriasis                       36
small_adenoma                   26
ibd_crohn_disease               25
large_adenoma                   13
overweight                      10
obese                            5
underweight                      1
Name: count, dtype: int64

In [155]:
classes = list(new_abundance_data['disease_new'].unique())

classes

['healthy',
 'obesity',
 'stec2-positive',
 'ibd_ulcerative_colitis',
 'ibd_crohn_disease',
 'psoriasis',
 'cirrhosis',
 'obese',
 'overweight',
 'underweight',
 't2d',
 'impaired_glucose_tolerance',
 'cancer',
 'small_adenoma',
 'large_adenoma']

In [156]:
# Multiclass classification

class MicrobiomeDataset(Dataset):
    def __init__(self, data, classes, device):
        self.data = data
        self.classes = classes
        self.device = device

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data.iloc[idx, :]
        sample_values = sample[ABUNDANCE_START:].values.astype(np.float32)
        sample_values = torch.from_numpy(sample_values).to(self.device)
        label = self.classes.index(sample['disease_new'])
        label = torch.tensor(label).to(self.device)
        return sample_values, label

In [157]:
# Dataloader

train_data = new_abundance_data.sample(frac=0.8, random_state=42)
test_data = new_abundance_data.drop(train_data.index)

train_dataset = MicrobiomeDataset(train_data, classes, device)
test_dataset = MicrobiomeDataset(test_data, classes, device)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("The number of samples in the training dataset is: ", len(train_data))
print("The number of samples in the test dataset is: ", len(test_data))

The number of samples in the training dataset is:  2888
The number of samples in the test dataset is:  722


In [158]:
# Model

class MicrobiomeClassifier(nn.Module):
    def __init__(self, input_size, output_size, layers):
        super(MicrobiomeClassifier, self).__init__()
        
        self.sequence = nn.Sequential()
        
        for i, layer in enumerate(layers):
            if i == 0:
                self.sequence.add_module(f'linear_{i}', nn.Linear(input_size, layer))
                self.sequence.add_module(f'relu_{i}', nn.ReLU())
            else:
                self.sequence.add_module(f'linear_{i}', nn.Linear(LAYERS[i-1], layer))
                self.sequence.add_module(f'relu_{i}', nn.ReLU())
                
        self.sequence.add_module('output', nn.Linear(layers[-1], output_size))
        self.sequence.add_module('softmax', nn.Softmax(dim=1))    
                
    def forward(self, x):
        return self.sequence(x)
    
    def print_model(self):
        print(self.sequence)

In [159]:
# Training

def train(model, data_loader, criterion, optimizer):
    """
    Generic training function
    """
    model.train() # Set model to training mode
    running_loss = 0.0
    for _, data in enumerate(data_loader):
        inputs, labels = data
        
        optimizer.zero_grad() # Zero the gradients
        
        outputs = model(inputs) # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward() # Backward pass
        optimizer.step() # Update weights
        
        running_loss += loss.item()
    return running_loss / len(data_loader)

def evaluate(model, data_loader, criterion):
    """
    Generic evaluation function    
    """
    model.eval() # Set model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for _, data in enumerate(data_loader):
            inputs, labels = data
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())
            
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return running_loss / len(data_loader), correct / total, true_labels, predicted_labels

In [160]:
model = MicrobiomeClassifier(input_size=train_data.shape[1] - ABUNDANCE_START, output_size=len(classes), layers=LAYERS).to(device)
model.print_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Sequential(
  (linear_0): Linear(in_features=3302, out_features=1024, bias=True)
  (relu_0): ReLU()
  (linear_1): Linear(in_features=1024, out_features=512, bias=True)
  (relu_1): ReLU()
  (linear_2): Linear(in_features=512, out_features=256, bias=True)
  (relu_2): ReLU()
  (linear_3): Linear(in_features=256, out_features=128, bias=True)
  (relu_3): ReLU()
  (linear_4): Linear(in_features=128, out_features=64, bias=True)
  (relu_4): ReLU()
  (output): Linear(in_features=64, out_features=15, bias=True)
  (softmax): Softmax(dim=1)
)


In [161]:
EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss = train(model, train_loader, criterion, optimizer)
    _, train_accuracy, true, pred = evaluate(model, train_loader, criterion)
    test_loss, test_accuracy, true_test, pred_test = evaluate(model, test_loader, criterion)
    print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}, Test Accuracy: {test_accuracy}')

Epoch 1/10, Train Loss: 2.095568673951285, Train Accuracy: 0.7479224376731302, Test Accuracy: 0.7368421052631579
Epoch 2/10, Train Loss: 2.0706234017571252, Train Accuracy: 0.7479224376731302, Test Accuracy: 0.7368421052631579
Epoch 3/10, Train Loss: 2.0685629582667087, Train Accuracy: 0.7479224376731302, Test Accuracy: 0.7368421052631579
Epoch 4/10, Train Loss: 2.0716536045074463, Train Accuracy: 0.7479224376731302, Test Accuracy: 0.7368421052631579
Epoch 5/10, Train Loss: 2.0685629674366544, Train Accuracy: 0.7479224376731302, Test Accuracy: 0.7368421052631579
Epoch 6/10, Train Loss: 2.0695931780469285, Train Accuracy: 0.7479224376731302, Test Accuracy: 0.7368421052631579
Epoch 7/10, Train Loss: 2.0685629425468024, Train Accuracy: 0.7479224376731302, Test Accuracy: 0.7368421052631579
Epoch 8/10, Train Loss: 2.0685629543367323, Train Accuracy: 0.7479224376731302, Test Accuracy: 0.7368421052631579
Epoch 9/10, Train Loss: 2.0695931754269443, Train Accuracy: 0.7479224376731302, Test Accu

In [170]:
# Confusion matrix

from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(true, pred, labels=np.arange(len(classes)))

print(conf_matrix.shape)

# write confusion matrix to file

conf_matrix_df = pd.DataFrame(conf_matrix, index=classes, columns=classes)
conf_matrix_df.to_csv(os.path.join(DATA_FOLDER, "confusion_matrix.csv"))

(15, 15)
